In [1]:
#!/usr/bin/python
# -*- coding: latin-1 -*-

import inspect, os, sys

try :
    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

except :
    current_folder = os.path.realpath(os.path.abspath(os.path.split(inspect.getfile(inspect.currentframe()))[0]))
    folder_parts = current_folder.split(os.sep)
    pywikibot_folder = os.sep.join(folder_parts[:-1])

    if current_folder not in sys.path:
        sys.path.insert(0, current_folder)
    if pywikibot_folder not in sys.path:
        sys.path.insert(0, pywikibot_folder)

    import pywikibot as pb
    from pywikibot.specialbots import UploadRobot

from bs4 import BeautifulSoup
import requests
from requests.compat import quote
from mako.template import Template
from PIL import Image, ImageFile
from openpyxl import load_workbook

import calendar
from io import BytesIO
import re

from modules.wmtools import is_commons_file, get_hash

In [2]:
commons_site = pb.Site("commons", "commons")

In [3]:
cwd = os.getcwd()

images_directory = os.path.join(cwd, 'images')
if not os.path.exists(images_directory):
    os.makedirs(images_directory)

In [4]:
EXCEL_FILE = "diario.madrid.es.xlsx"
wb = load_workbook(os.path.join(cwd, EXCEL_FILE))

In [24]:
template = u"""=={{int:filedesc}}==
{{Information
|description={{es|1=${description}}}
|date=${date}
|source=[${url} Diario de Madrid - ${title}]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discasto]]
[[Category:Vueltas de San Antón]]"""

In [25]:
def reduce_name (title) :
    if len(title) > 200 :
        title_tokens = title.split('(')
        if len (title_tokens) > 2 :
            title_lead = '('.join([title_tokens[i] for i in range (len(title_tokens)-1)]).strip()
            title_end = title_tokens[-1]
        else :
            title_lead = title_tokens[0].strip()
            title_end = title_tokens[1]

        title_lead = title_lead[:189]
        title_lead_tokens = title_lead.split(' ')
        title_lead_tokens = [title_lead_tokens[i] for i in range (len(title_lead_tokens)-1)]
        title_lead = ' '.join(title_lead_tokens)
        title_lead = title_lead + '...'
        title = '%s (%s' %(title_lead, title_end)
    title = title.replace(':', ',')
    return title

In [26]:
onlyfiles = [os.path.join(images_directory, f) for f in os.listdir(images_directory) if os.path.isfile(os.path.join(images_directory, f))]
for i, image in enumerate(onlyfiles):
    sha = get_hash(image)
    if is_commons_file (sha) == False:
        print (image)
    else :
        #print (image)
        os.remove(image)

D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Carmena, Maestre y Castaño asisten a la lectura del pregón de las fiestas de San Antón 01.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Carmena, Maestre y Castaño asisten a la lectura del pregón de las fiestas de San Antón 02.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Carmena, Maestre y Castaño asisten a la lectura del pregón de las fiestas de San Antón 03.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Carmena, Maestre y Castaño asisten a la lectura del pregón de las fiestas de San Antón 04.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Madrid, una ciudad respetuosa con los animales 01.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Madrid, una ciudad respetuosa con los animales 02.jpg
D:\Datos\Miguel-Angel\Code\Python\core\wmcommons-helpers\images\Madrid, una ciudad respetuosa con los animales 03.jpg
D:\Datos\Migue

In [27]:
used_names = []

ws = wb[u'Sheet']
for row in ws.iter_rows():
    image_path = row[0].value
    image_name = image_path.split('\\')[-1]
    # The image does not exist. Skipping
    if os.path.isfile(image_path) == False :
        print ("*Image name ({0}) deleted from local directory".format(image_name))
        continue
    else :
        print ("** Handling image {}".format(image_name))

    # If the image name is already in commons, find a new name
    if pb.Page(commons_site, image_name, ns=6).exists():
        print ("**Image name ({0}) already used in Commons".format(image_name))
        used_names.append(image_name)

    while True:
        if image_name in used_names :
            # Finding a new name
            image_subject = '.'.join(image_name.split('.')[:-1])
            image_extension = 'jpg'
            p = re.compile('(.*) ([0-9]{2}\.jpg)')
            m = p.match(image_name)
            if m is None:
                image_name = image_subject + ' 01.' + image_extension
            else :
                counter = int(m.group(2)[:2]) + 1
                image_name = '{} {:02d}.{}'.format(m.group(1), counter, image_extension)

            if pb.Page(commons_site, image_name, ns=6).exists():
                print ("Image name ({0}) already used in Commons. Finding a new name".format(image_name))
                used_names.append(image_name)
        else :
            print ("Preparing to upload image with name {0}".format(image_name))
            used_names.append(image_name)
            break

    # image upload
    vars = {
            "url": row[3].value,
            "description": row[7].value,
            "year": row[5].value,
            "month": row[6].value,
            "date": row[4].value,
            "title": row[2].value
            }
    t = Template(template)
    _text = t.render(**vars)

    bot = UploadRobot([image_path],
                      description = _text,
                      useFilename = image_name,
                      keepFilename = True,
                      verifyDescription = False,
                      ignoreWarning = True,
                      targetSite = commons_site)
    bot.run()

    os.remove(image_path)

*Image name (Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 01.jpg) deleted from local directory
*Image name (Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 02.jpg) deleted from local directory
*Image name (Higueras recibe a los nuevos alumnos de la residencia de San Ildefonso 03.jpg) deleted from local directory
*Image name (Annie Hart en Chamberí, fotografía humanitaria en CentroCentro y un paseo guiado por Caño Roto 01.jpg) deleted from local directory
*Image name (Annie Hart en Chamberí, fotografía humanitaria en CentroCentro y un paseo guiado por Caño Roto 02.jpg) deleted from local directory
*Image name (Annie Hart en Chamberí, fotografía humanitaria en CentroCentro y un paseo guiado por Caño Roto 03.jpg) deleted from local directory
*Image name (Annie Hart en Chamberí, fotografía humanitaria en CentroCentro y un paseo guiado por Caño Roto 04.jpg) deleted from local directory
*Image name (Annie Hart en Chamberí, fotografía humanitar

Preparing to upload image with name Madrid, una ciudad respetuosa con los animales 01.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=“Apostamos por continuar la gran tradición de San Antón y porque la biosostenibilidad sea posible en Madrid, que sea posible la convivencia entre la ciudad y la naturaleza, la ciudad y el medioambiente”. Con estas palabras, la alcaldesa Manuela Carmena, se ha dirigido al público, acompañada por el padre Ángel, el delegado de Economía y Hacienda y concejal del distrito de Centro, Jorge García Castaño; la portavoz del Gobierno municipal, Rita Maestre; y la concejala de Carabanchel y Chamberí, Esther Gómez, tras asistir a la lectura del pregón, que ha corrido a cargo de representantes de Ecologistas en Acción que impulsaron el proyecto de Renaturalización del Manzanares, Santiago Martín Barajas y Maria Ángeles Nieto.}}
|date=2018-01-13
|source=[https://diario.madrid.es/blog/notas-de-prensa/madrid-una-ciudad-respetuosa-con-los-animales/ Diario de Madrid - Madrid, una ciudad respetuosa con los animales]
|au

** Handling image Madrid, una ciudad respetuosa con los animales 02.jpg
Preparing to upload image with name Madrid, una ciudad respetuosa con los animales 02.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=“Apostamos por continuar la gran tradición de San Antón y porque la biosostenibilidad sea posible en Madrid, que sea posible la convivencia entre la ciudad y la naturaleza, la ciudad y el medioambiente”. Con estas palabras, la alcaldesa Manuela Carmena, se ha dirigido al público, acompañada por el padre Ángel, el delegado de Economía y Hacienda y concejal del distrito de Centro, Jorge García Castaño; la portavoz del Gobierno municipal, Rita Maestre; y la concejala de Carabanchel y Chamberí, Esther Gómez, tras asistir a la lectura del pregón, que ha corrido a cargo de representantes de Ecologistas en Acción que impulsaron el proyecto de Renaturalización del Manzanares, Santiago Martín Barajas y Maria Ángeles Nieto.}}
|date=2018-01-13
|source=[https://diario.madrid.es/blog/notas-de-prensa/madrid-una-ciudad-respetuosa-con-los-animales/ Diario de Madrid - Madrid, una ciudad respetuosa con los animales]
|au

...

The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=Como viene siendo tradicional, los animales serán los protagonistas de las primeras fiestas patronales del año. Con el epicentro de las celebraciones en la Iglesia de San Antón del distrito de Centro, en esta edición el Ayuntamiento de Madrid ha extendido las actividades animalistas al resto de barrios de Madrid, convirtiendo la ciudad en su conjunto en capital de la defensa y promoción de los derechos de los animales.}}
|date=2018-01-09
|source=[https://diario.madrid.es/blog/notas-de-prensa/san-anton-2018-se-extiende-por-toda-la-ciudad-para-manifestar-el-compromiso-de-madrid-con-el-bienestar-animal/ Diario de Madrid - San Antón 2018 se extiende por toda la ciudad para manifestar el compromiso de Madrid con el bienestar animal]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discast

** Handling image San Antón 2018 se extiende por toda la ciudad para manifestar el compromiso de Madrid con el bienestar animal 02.jpg
Preparing to upload image with name San Antón 2018 se extiende por toda la ciudad para manifestar el compromiso de Madrid con el bienestar animal 02.jpg


The suggested description is:
=={{int:filedesc}}==
{{Information
|description={{es|1=Como viene siendo tradicional, los animales serán los protagonistas de las primeras fiestas patronales del año. Con el epicentro de las celebraciones en la Iglesia de San Antón del distrito de Centro, en esta edición el Ayuntamiento de Madrid ha extendido las actividades animalistas al resto de barrios de Madrid, convirtiendo la ciudad en su conjunto en capital de la defensa y promoción de los derechos de los animales.}}
|date=2018-01-09
|source=[https://diario.madrid.es/blog/notas-de-prensa/san-anton-2018-se-extiende-por-toda-la-ciudad-para-manifestar-el-compromiso-de-madrid-con-el-bienestar-animal/ Diario de Madrid - San Antón 2018 se extiende por toda la ciudad para manifestar el compromiso de Madrid con el bienestar animal]
|author=[https://diario.madrid.es/ Diario de Madrid]
|permission=
|other versions=
}}

=={{int:license-header}}==
{{Diario de Madrid}}

[[Category:Files uploaded by User:Discast

*Image name (Thriller teatral en el Fernán Gómez, fotos de autora en Arganzuela y Gatos de Estambul en Cineteca 01.jpg) deleted from local directory
*Image name (Thriller teatral en el Fernán Gómez, fotos de autora en Arganzuela y Gatos de Estambul en Cineteca 02.jpg) deleted from local directory
*Image name (Thriller teatral en el Fernán Gómez, fotos de autora en Arganzuela y Gatos de Estambul en Cineteca 03.jpg) deleted from local directory
*Image name (Una segunda oportunidad para los abetos navideños 01.jpg) deleted from local directory
*Image name (El Ayuntamiento recomienda el transporte público para ir al centro 01.jpg) deleted from local directory
*Image name (Niños y niñas de la Cañada Real reciben regalos de los Magos 01.jpg) deleted from local directory
*Image name (Niños y niñas de la Cañada Real reciben regalos de los Magos 02.jpg) deleted from local directory
*Image name (Niños y niñas de la Cañada Real reciben regalos de los Magos 03.jpg) deleted from local directory
*Im

Upload successful.
Upload of San Antón 2018 se extiende por toda la ciudad para manifestar el compromiso de Madrid con el bienestar animal 02.jpg successful.

1 pages read
1 pages written
Script terminated successfully.
